In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load sph_dynamic.csv into a Pandas dataframe and display the first 5 rows of the data
dynamic_data = pd.read_csv('data/sph_dynamic.csv')
dynamic_data.head()

,stay_id,charttime,total_protein,calcium,creatinine,glucose,sodium,chloride,heart_rate,sbp,...,ph,lactate,pt,urineoutput,sofa_respiration,sofa_coagulation,sofa_liver,sofa_cardiovascular,sofa_cns,sofa_renal
0,35715575,2148-12-27 18:15:00.000,NaN,8.5,0.9,137.0,138.0,104.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34483718,2118-01-04 03:58:00.000,NaN,8.2,0.8,129.0,141.0,101.0,NaN,NaN,...,NaN,NaN,12.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31826892,2163-03-10 19:59:00.000,NaN,7.7,0.4,112.0,136.0,98.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36154799,2131-12-02 19:14:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32732521,2116-08-12 12:45:00.000,NaN,NaN,4.0,135.0,139.0,105.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Get the number of missing values in each column of the dynamic_data
dynamic_data.isnull().sum()

stay_id                   0
charttime                 0
total_protein          6930
calcium                 933
creatinine              261
glucose                 444
sodium                  214
chloride                241
heart_rate             6833
sbp                    6895
dbp                    6895
mbp                    6887
resp_rate              6832
temperature            6974
hemoglobin             1179
wbc                    1207
alt                    3964
ast                    3936
alp                    3976
bilirubin_total        3957
bilirubin_direct       6808
bilirubin_indirect     6812
ph                     7004
lactate                7012
pt                     3068
urineoutput            6942
sofa_respiration       7005
sofa_coagulation       7023
sofa_liver             7023
sofa_cardiovascular    6872
sofa_cns               6979
sofa_renal             7024
dtype: int64

In [4]:
# Drop the columns in dynamic_data with more than 80% of the values
for col in dynamic_data.columns:
    if dynamic_data[col].isnull().sum() > len(dynamic_data)*0.8:
        del dynamic_data[col]

In [5]:
# Now check the number of missing values in each column of the dynamic_data again
dynamic_data.isnull().sum()

stay_id               0
charttime             0
calcium             933
creatinine          261
glucose             444
sodium              214
chloride            241
hemoglobin         1179
wbc                1207
alt                3964
ast                3936
alp                3976
bilirubin_total    3957
pt                 3068
dtype: int64

In [6]:
# ['alt','ast','alp','bilirubin_total','pt'] are liver function related test results
# create a new binary column 'liver_function_test', True/1 means have ever taken liver function test
liver_test_result = ['alt','ast','alp','bilirubin_total','pt']
def liver_categorize(group):
    flag = True
    for i in liver_test_result:
        if group[i].notnull().any():
            flag = False
    if flag:
        group['liver_function_test'] = False
    else:
        group['liver_function_test'] = True
    return group

dynamic_data = dynamic_data.groupby('stay_id').apply(liver_categorize)

/tmp/ipykernel_358063/240455136.py:15: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dynamic_data = dynamic_data.groupby('stay_id').apply(liver_categorize)


Assuming that patients with no relevant results recorded do not have liver issues, we impute these patients' missing values of these columns with random number in normal range
Note: but i can not find the unit and normal range for them so i drop them first >_<

In [7]:
dynamic_data.isnull().sum()

stay_id                   0
charttime                 0
calcium                 933
creatinine              261
glucose                 444
sodium                  214
chloride                241
hemoglobin             1179
wbc                    1207
alt                    3964
ast                    3936
alp                    3976
bilirubin_total        3957
pt                     3068
liver_function_test       0
dtype: int64

In [8]:
# use KNN to impute the rest
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 10)
dynamic_data.iloc[:,2:] = imputer.fit_transform(dynamic_data.iloc[:,2:])

In [9]:
# define normal ranges for each column
normal_ranges = {
    'alt': (5, 40),
    'ast': (10, 35),
    'alp': (40, 130),
    'bilirubin_total': (0.1, 1.0),
    'pt': (9.5, 13.5)
}

In [10]:
mask = dynamic_data['liver_function_test'] == False
n_no_test = mask.sum()
def sample_normal(col):
    lower = normal_ranges[col][0]
    upper = normal_ranges[col][1]
    return np.random.normal(loc=(lower+upper)/2, scale=(upper-lower)/6, size=n_no_test)

sampled_alt = sample_normal("alt")
sampled_ast = sample_normal("ast")
sampled_alp = sample_normal("alp")
sampled_bilirubin_total = sample_normal("bilirubin_total")
sampled_pt = sample_normal("pt")
dynamic_data.loc[mask, 'alt'] = sampled_alt
dynamic_data.loc[mask, 'ast'] = sampled_ast
dynamic_data.loc[mask, 'alp'] = sampled_alp
dynamic_data.loc[mask, 'bilirubin_total'] = sampled_bilirubin_total
dynamic_data.loc[mask, 'pt'] = sampled_pt

In [11]:
# To address the issue that same patient has differrent results at the same charttime
dynamic_data = dynamic_data.groupby(['stay_id','charttime']).mean().reset_index()

In [12]:
# store the new dynamic_data into a csv file
dynamic_data.to_csv('preprocessed_dynamic_data.csv', index=False)

# Make a copy of the dynamic_data
new_dynamic_data = dynamic_data.copy()

In [13]:
# Get the number of patients for each number of charttime
# and print the data out in this format: number of charttime, number of patients
new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts().sort_index()

# Print the number of charttimes for each patient
# dynamic_data.groupby('stay_id')['charttime'].count()

# Print the total number of unique patients
# print('Total number of unique patients: ', dynamic_data['stay_id'].nunique())

1      621
2      648
3      263
4      102
5       51
6       40
7       28
8       22
9       26
10      19
11      12
12      10
13       9
14       4
15       3
16       3
17       9
18       3
19       3
20       5
21       1
22       4
23       2
24       2
25       3
26       1
27       1
28       8
29       2
30       1
31       1
33       2
38       1
39       1
40       1
41       2
44       1
45       1
49       1
61       1
62       1
82       1
93       1
131      1
196      1
Name: charttime, dtype: int64

In [14]:
# Look through the list and find the patients with exactly 4 charttimes
print('Patients with exactly 4 charttimes: ', new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts()[4])

# Print the number of patients with less than 4 charttimes
print('Patients with less than 4 charttimes: ', new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts().loc[:3].sum())

# Print the number of patients with more than 4 charttimes
print('Patients with more than 4 charttimes: ', new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts().loc[5:].sum())

# Create two empty lists: one for patients with less than 4 charttimes and one for patients with more than 4 charttimes
stay_id_less_than_4 = []
stay_id_more_than_4 = []

# Loop through dynamic_data
for stay_id, group in new_dynamic_data.groupby('stay_id'):
    # If a stay_id has less than 4 charttimes, append the stay_id to stay_id_less_than_4
    if len(group) < 4:
        stay_id_less_than_4.append(stay_id)
    # Else if a stay_id has more than 4 charttimes, append the stay_id to stay_id_more_than_4
    elif len(group) > 4:
        stay_id_more_than_4.append(stay_id)


Patients with exactly 4 charttimes:  102
Patients with less than 4 charttimes:  1532
Patients with more than 4 charttimes:  289


In [15]:
# For patients with less than 4 charttimes, pad the data with the last charttime to make sure each patient has 4 charttimes
# For patients with more than 4 charttimes, use the sliding window method to get 4 charttimes

# Get the list of charttimes less than 4
less_than_4 = new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts().sort_index().index[:3]
print('Number of charttime counts less than 4: ', len(less_than_4))

# Get the list charttime counts greater than 4
more_than_4 = new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts().sort_index().index[4:]
print('Number of charttimes counts greater than 4: ', len(more_than_4))

count = 0

# Use the last charttime to pad the data for patients with less than 4 charttimes
for i in less_than_4:
    count += 1
    print(count)
    mask = new_dynamic_data.groupby('stay_id')['charttime'].count() == i
    # pad the data for each patient stay_id
    for j in stay_id_less_than_4:
        for k in range(4-i):
            new_dynamic_data = new_dynamic_data.append(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].iloc[-1], ignore_index=True)

count = 0

# # Use the sliding window method to get 4 charttimes
# # Only use the 4 charttimes obtained from the sliding window method for each patient with more than 4 charttimes
# # Drop the rest of the charttimes for each patient with more than 4 charttimes
# for i in more_than_4:
#     count += 1
#     print(count)
#     mask = new_dynamic_data.groupby('stay_id')['charttime'].count() == i
#     # For each patient stay_id, use the sliding window method to get 4 charttimes
#     # Add these 4 charttimes to new_dynamic_data
#     # Drop the original charttimes for each patient stay_id
#     for j in stay_id_more_than_4:
#         for k in range(i-3):
#             new_dynamic_data = new_dynamic_data.append(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].iloc[k:k+4], ignore_index=True)
#         new_dynamic_data = new_dynamic_data.drop(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].index)


# Check if the number of patients with 4 charttimes is equal to the total number of patients
print('Number of patients with 4 charttimes: ', new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts()[4])

Number of charttime counts less than 4:  3
Number of charttimes counts greater than 4:  41
1


/tmp/ipykernel_358063/1164677254.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dynamic_data = new_dynamic_data.append(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].iloc[-1], ignore_index=True)


2


/tmp/ipykernel_358063/1164677254.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dynamic_data = new_dynamic_data.append(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].iloc[-1], ignore_index=True)


3


...

40


/tmp/ipykernel_358063/1164677254.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dynamic_data = new_dynamic_data.append(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].iloc[k:k+4], ignore_index=True)


41


/tmp/ipykernel_358063/1164677254.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dynamic_data = new_dynamic_data.append(new_dynamic_data.loc[new_dynamic_data['stay_id'] == j].iloc[k:k+4], ignore_index=True)


Number of patients with 4 charttimes:  102


In [16]:
# Get the number of patients for each number of charttime
# and print the data out in this format: number of charttime, number of patients
new_dynamic_data.groupby('stay_id')['charttime'].count().value_counts().sort_index()

4    102
7    621
8    648
9    263
Name: charttime, dtype: int64

In [ ]:
# Load static_data
static_data = pd.read_csv('data/static_data.csv')

# Merge static_data and dynamic_data
merged_data = pd.merge(static_data, dynamic_data, on='stay_id')

### Design a time-series classification model using LSTM RNN + Single Layer Perceptron (SLP) classifier

In [40]:
# Import keras and tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, Concatenate
from tensorflow.keras.models import Model

# Define timesteps and the number of features
n_timesteps = 8
n_features = 7

# RNN + MLP Model

# Define input layer
recurrent_input = Input(shape=(n_timesteps,n_features),name=TIMESERIES_INPUT)
static_input = Input(shape=(x_train_over_static.shape[1], ),name=STATIC_INPUT)

# RNN Layers
# layer - 1
rec_layer_one = Bidirectional(LSTM(128, 
                              kernel_regularizer=l2(0.01),
                              recurrent_regularizer=l2(0.01),
                              return_sequences=True),
                              name=BIDIRECTIONAL_LAYER_1)(recurrent_input)
rec_layer_one = Dropout(0.1,name=DROPOUT_LAYER_1)(rec_layer_one)

# layer - 2
rec_layer_two = Bidirectional(LSTM(64, 
                              kernel_regularizer=l2(0.01),
                              recurrent_regularizer=l2(0.01)),
                              name =BIDIRECTIONAL_LAYER_2)(rec_layer_one)
rec_layer_two = Dropout(0.1,name=DROPOUT_LAYER_2)(rec_layer_two)

# SLP Layers
static_layer_one = Dense(64, kernel_regularizer=l2(0.001), activation='relu',name=DENSE_LAYER_1)(static_input)

# Combine layers - RNN + MLP
combined = Concatenate(axis= 1, name=CONCATENATED_TIMESERIES_STATIC)([rec_layer_two, static_layer_one])
combined_dense_two = Dense(64, activation='relu', name=DENSE_LAYER_2)(combined)
output = Dense(n_outputs, activation='sigmoid', name=OUTPUT_LAYER)(combined_dense_two)

# Compile Model
model = Model(inputs=[recurrent_input,static_input],outputs=[output])

# binary cross entropy loss
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])

# focal loss
def focal_loss_custom(alpha, gamma):
    def binary_focal_loss(y_true, y_pred):

        fl = tfa.losses.SigmoidFocalCrossEntropy(alpha=alpha, gamma=gamma)

        y_true_K = K.ones_like(y_true)

        focal_loss = fl(y_true, y_pred)

        return focal_loss
return binary_focal_loss

model.compile(loss=focal_loss_custom(alpha=0.2, gamma=2.0), optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])

model.summary()


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Define metrics for evaluating the model - recall, precision and f1-score
def recall_m(y_true, y_pred):
   true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
   possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
   recall = true_positives / (possible_positives + K.epsilon())
   return recall
def precision_m(y_true, y_pred):
   true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
   predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
   precision = true_positives / (predicted_positives + K.epsilon())
   return precision
def f1_m(y_true, y_pred):
   precision = precision_m(y_true, y_pred)
   recall = recall_m(y_true, y_pred)
   return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# fit network
history =  model.fit([np.asarray(x_train_reshape).astype('float32'), np.asarray(x_train_over_static).astype('float32')],
                     y_train_reshape, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=([np.asarray(x_val_reshape).astype('float32'), np.asarray(x_val_static).astype('float32')],y_val_reshape))
# summarize history for accuracy
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper left')
pyplot.show()
# summarize history for loss
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper left')
pyplot.show()
#evaluate model
loss, accuracy, f1_score, precision, recall = model.evaluate([np.asarray(x_test_reshape).astype('float32'),np.asarray(x_test_static).astype('float32')], y_test_reshape, batch_size=batch_size, verbose=0)
#print output
print("Accuracy:{} , F1_Score:{}, Precision:{}, Recall:{}".format(accuracy, f1_score, precision, recall))